# Scraping the Dextoools trade data

In [3]:
import requests
from bs4 import BeautifulSoup

# specify the URL of the WETH/USDT page
url = 'https://www.dextools.io/app/en/ether/pair-explorer/0x4e68ccd3e89f51c3074ca5072bbac773960dfa36'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

# send a GET request to the URL
response = requests.get(url, headers=headers)

# parse the HTML content of the page using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# find the main content of the page by looking for the first <div> tag with class 'mw-parser-output'
div_table_frame = soup.find("div", {"class":"ng-tns-c167-3"})
# content_div = div_content_text.find('div', {'class': 'mw-parser-output'})

# # get all the text content from the page by concatenating the text of all <p> tags within the content div
# text_content = ''
# for p in content_div.find_all('p'):
#     text_content += p.get_text()

# # print the text content
# print(text_content)

In [2]:
soup.find_all("p")##hmm this again produce error, continue here

[<p>You must have JavaScript enabled to run this app!</p>]

In [2]:
import requests
from bs4 import BeautifulSoup
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# specify the URL of the WETH/USDT page
url = 'https://www.dextools.io/app/en/ether/pair-explorer/0x811beed0119b4afce20d2583eb608c6f7af1954f'

options = Options()
#options.add_experimental_option("detach", True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=options)

driver.get(url)
#driver.maximize_window()

#wait = WebDriverWait(driver, 15).until(lambda driver: driver.find_element("xpath", "//div[@role='table']"))#.is_displayed())

# table = driver.find_elements("xpath", "//div[@role='table']")
# for i in table:
#     print(i.get_attribute("innerHTML"))
# table = driver.find_element("xpath", "//datatable-body[1]")
# print(table.get_attribute("innerHTML"))


# I suppose this is one way, get the content into soup and work on it from there
# content = driver.page_source
# soup = BeautifulSoup(content, 'html.parser')
# print(soup)

# The Stack overflow way
# get table
tableElement = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, 'ngx-datatable'))
)

# scroll into table view
driver.execute_script("arguments[0].scrollIntoView();", tableElement)

#probably here I should check no bots!

# scrolling through the table body to the bottom
tableBodyelement = tableElement.find_element(By.TAG_NAME, "datatable-body-cell")#   .find_element_by_tag_name('datatable-body-cell')
driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", tableBodyelement)


rowWrapper = tableElement.find_elements(By.TAG_NAME, 'datatable-row-wrapper')

#for i in rowWrapper:
#    print(i.get_attribute("innerHTML"))


for row in rowWrapper:
    cells = row.find_elements(By.TAG_NAME, 'datatable-body-cell')
    date = cells[0].text
    type = cells[1].text
    price_usd = cells[2].text
    price_eth = cells[3].text
    amount_coin_1 = cells[4].text
    total_coin_2 = cells[5].text  #the naming here should be done better, it will depend on the pairs I will be trading
    total_chain_coin = cells[6].text
    maker = cells[7].find_element(By.TAG_NAME, 'a').get_attribute('href')
    print(date, type, price_usd, price_eth, amount_coin_1, total_coin_2, total_chain_coin, maker)
    print('----')    

#print(tableElement.get_attribute("innerHTML"))
#driver.quit()


#next continue here. Tydy up the code, insert the whole tyhing into while loop, save into dataframe.
#Also TODO: filter bot activity



2023-03-18 11:26:11 sell $0.00001113 0.0₈6139 3,499,873 0.0214 0x2d...6946 https://etherscan.io/tx/0xd8bf40ad104b855bafab085912410ad5764a086dee79b0a9c7a244beb81bbf36
----
2023-03-18 11:24:47 sell $0.00001114 0.0₈6143 34,830,923 0.2139 0x00...f446 https://etherscan.io/tx/0xa11fc2abefc69ae0c77dae359ab6b5d2b16fe8d1a20b694caa9a2911d0c36c10
----
2023-03-18 11:05:59 buy $0.00001121 0.0₈6183 7,665,340 0.0474 0x56...f663 https://etherscan.io/tx/0x82343d92094761032a14c5fad106a120bbb87c9014581e83fdae0e124b83a6e6
----
2023-03-18 10:46:23 sell $0.00001113 0.0₈6146 14,233,428 0.0874 0x6d...94f2 https://etherscan.io/tx/0x7b0c34fe4d66f94d2e90737ac4415619e446434b927a3b3e8229abfa8c378512
----
2023-03-18 10:39:47 buy $0.00001117 0.0₈6184 6,633,838 0.0410 0xf0...67bf https://etherscan.io/tx/0xf91b51b435900e9d322d7cb6b6a834b8692b8122a70a6802ccb7668bf2743798
----
2023-03-18 10:27:35 buy $0.00001117 0.0₈6183 3,128,241 0.0193 0xb5...b3a9 https://etherscan.io/tx/0x05057c8146a3b9637eab3fdad7b6bb6f2760826b98201

In [38]:
#now the way with while loop for more pages
# specify the URL of the WETH/USDT page
url = 'https://www.dextools.io/app/en/ether/pair-explorer/0x811beed0119b4afce20d2583eb608c6f7af1954f'

options = Options()
#options.add_experimental_option("detach", True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=options)

driver.get(url)

page = 0

all_results = []  # list for all rows

#apply filtering for no bots
filter_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//a[@class="d-inline-block wallet-filter-modal ng-tns-c168-3"]'))
)
#filter_button = driver.find_element(By.XPATH, '//a[@class="d-inline-block wallet-filter-modal ng-tns-c168-3"]')
driver.execute_script("arguments[0].click();", filter_button) #opening filter window
#time.sleep(0.5) #for some reason this is important?

bot_checkbox = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//input[@id="bot-filtered"]'))
)
#bot_checkbox = driver.find_element(By.XPATH, '//input[@id="bot-filtered"]')
driver.execute_script("arguments[0].click();", bot_checkbox) #ticking the checkbox
#time.sleep(0.5)

apply_filter_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//button[@class="btn btn-success ng-star-inserted"]'))
)
#apply_filter_button = driver.find_element(By.XPATH, '//button[@class="btn btn-success ng-star-inserted"]')
driver.execute_script("arguments[0].click();", apply_filter_button) #applying filter
time.sleep(0.5)

while True:

    page += 1
    print('--- page:', page, '---')
    print("before scrolling to view")
    # get table
    tableElement = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, 'ngx-datatable'))
    )
    # scroll into table view
    driver.execute_script("arguments[0].scrollIntoView();", tableElement)

    # scrolling through the table body to the bottom
    tableBodyelement = tableElement.find_element(By.TAG_NAME, "datatable-body-cell")
    driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", tableBodyelement)

    print("after scrolling to view")
    




    #trying faster solution by chatGPT
    # rowWrapper = tableElement.find_elements(By.TAG_NAME, 'datatable-row-wrapper')
    # print("finding elements")
    # for row in rowWrapper:
    #     cells = row.find_elements(By.TAG_NAME, 'datatable-body-cell')
    #     date = cells[0].text
    #     type = cells[1].text
    #     price_usd = cells[2].text
    #     price_eth = cells[3].text
    #     amount_coin_1 = cells[4].text
    #     total_coin_2 = cells[5].text  #the naming here should be done better, it will depend on the pairs I will be trading
    #     total_chain_coin = cells[6].text
    #     maker = cells[7].find_element(By.TAG_NAME, 'a').get_attribute('href')

    #     # if page%10 == 0:
    #     #     print(date, type, price_usd, price_eth, amount_coin_1, total_coin_2, total_chain_coin, maker)
    #     #     print('----')  
        
    #     # add row to list
    #     all_results.append( [date, type, price_usd, price_eth, amount_coin_1, total_coin_2, total_chain_coin, maker] )
    rowWrapper = tableElement.find_elements(By.TAG_NAME, 'datatable-row-wrapper')
    print("finding elements")
    all_results += [[cells[0].text,
                    cells[1].text,
                    cells[2].text,
                    cells[3].text,
                    cells[4].text,
                    cells[5].text,
                    cells[6].text,
                    cells[7].find_element(By.TAG_NAME, 'a').get_attribute('href')
    ] for row in rowWrapper for cells in [row.find_elements(By.TAG_NAME, 'datatable-body-cell')]]



    ######
                                
    print("elements found")
    try:
        next_page = driver.find_element(By.XPATH, '//a[@aria-label="go to next page"]')

        #verify that the button is not disabled
        next_page_parent = next_page.find_element(By.XPATH, '..')
        if "disabled" in next_page_parent.get_attribute("class"):
            print("End of table")
            break

        #click on the button and move to next page
        print("clicking next page")
        driver.execute_script("arguments[0].click();", next_page) #this should solve the issue of not finding the element?
        time.sleep(0.5)
        print("next page clicked")
    except Exception as ex:
       print("Some error occured")
       break
    
driver.quit()    
# after loop convert to DataFrame and write it to excel


#print(all_results)
#import pandas as pd

#df = pd.DataFrame(all_results, columns=['date', 'type', 'price_usd', 'price_eth', 'amount_cuminu', 'total_eth', 'maker'])
#df.to_excel('results.xlsx')

# ---


#note, TODO next: it scrapes well, just at the end there is a problem that it cycles and runs the same set of table rows in a loop. 
# Resolve next time, look how the button changes at the end, if not available - perhaps if block?


--- page: 1 ---
before scrolling to view
after scrolling to view
finding elements
elements found
clicking next page
next page clicked
--- page: 2 ---
before scrolling to view
after scrolling to view
finding elements
elements found
clicking next page
next page clicked
--- page: 3 ---
before scrolling to view
after scrolling to view
finding elements
elements found
clicking next page
next page clicked
--- page: 4 ---
before scrolling to view
after scrolling to view
finding elements
elements found
clicking next page
next page clicked
--- page: 5 ---
before scrolling to view
after scrolling to view
finding elements
elements found
clicking next page
next page clicked
--- page: 6 ---
before scrolling to view
after scrolling to view
finding elements
elements found
clicking next page
next page clicked
--- page: 7 ---
before scrolling to view
after scrolling to view
finding elements
elements found
clicking next page
next page clicked
--- page: 8 ---
before scrolling to view
after scrolling to vi

In [39]:
print(len(all_results))

for i in all_results:
    print(i)

1500
['2023-03-18 16:17:11', 'buy', '$0.00001115', '0.0₈6168', '24,390,449', '0.1504', '0x05...c85b', 'https://etherscan.io/tx/0x0a543e5ca6d288505a5838f0b34c015614c2e9a85d2d17402005d8439da02ffe']
['2023-03-18 16:14:23', 'sell', '$0.00001108', '0.0₈6132', '32,227,072', '0.1976', '0xa7...4957', 'https://etherscan.io/tx/0x2bc43e1ce82dc92b76337fa5235c65d2ac6dde2834e93378b384dfd4323664e9']
['2023-03-18 16:10:11', 'sell', '$0.00001107', '0.0₈6138', '30,000,000', '0.1841', '0xa7...4957', 'https://etherscan.io/tx/0x2906a038e1ce5f71e741abe86378ed0482cfca7a7f746c1c5c9550fa74ae3709']
['2023-03-18 16:10:11', 'sell', '$0.00001107', '0.0₈6142', '12,834,183', '0.0788', '0xc3...e8ab', 'https://etherscan.io/tx/0xcc23d8bb81b35fa039f4eed01b994e393dd1feae7eef6f7fa6edcf59c2498ae0']
['2023-03-18 16:05:59', 'sell', '$0.00001111', '0.0₈6145', '15,000,306', '0.0921', '0x6c...c516', 'https://etherscan.io/tx/0x9f7a6a90fd61767d8680d92b18cb38c022650566304f4431f79d8f557142efb9']
['2023-03-18 16:05:11', 'sell', '$0.

In [24]:
#Just try to find an element and click it
# specify the URL of the WETH/USDT page
url = 'https://www.dextools.io/app/en/ether/pair-explorer/0x4e68ccd3e89f51c3074ca5072bbac773960dfa36'

options = Options()
#options.add_experimental_option("detach", True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=options)

driver.get(url)
time.sleep(5)
next_page = driver.find_element(By.XPATH, '//a[@aria-label="go to next page"]')



print(next_page.get_attribute("innerHTML"))
#next_page.click()
driver.execute_script("arguments[0].click();", next_page)
#print(test_elemnt.get_attribute("innerHTML"))



<i class="datatable-icon-right"></i>
